# Documentation 
<div align='right'> 12/31/2022 </div>

1. Initial setting 

In [32]:
import os
import sys
import math 
import warnings 

sys.path.insert(0,os.getcwd()+'\\data')
sys.path.insert(0,os.getcwd()+'\\models')
from simulator import simulator
from wave_detector import monowave

warnings.filterwarnings('ignore')

In [166]:
data_path=os.path.join(os.getcwd(),'data/btcusd.csv')
sim=simulator(data_path)

In [167]:
sim(5570)

#### Suppose new price is inputted to the monowave detector when the price is generated.
***
- Each price will be added to the price list and our indexing algorithm will save the status, idx number, monowave label of the price.

In [168]:
price=list()    # this is monowave price list
idx=1           # this is the number of current monowave 
status=str()    # it will be 'inc' or 'dec'
label_li=list()    # this is price's monowave label list

In [169]:
# how to add each price to the price list
price.append(sim.gen())

In [170]:
print(price)

[30598.0]


- Monowave Indexing <br>
: we can identify initial status after the number of price is at least two. 

In [171]:
if len(price)<3:
    label='M'+str(idx)
    if len(price)==2:
        if price[0] < price[-1]:
            status='inc'
        else:
            status='dec'
    else:
        print('we are here')
        pass
label_li.append(label)

we are here


In [172]:
#after add new price 
price.append(sim.gen())

#now price length is 2
if len(price)<3:
    label='M'+str(idx)
    if len(price)==2:
        if price[0] < price[-1]:
            status='inc'
        else:
            status='dec'
        print('we are here now')
    else:
        print('we are here')
        pass
label_li.append(label)

we are here now


In [173]:
# we can check all stuffs like price, idx, label, status
print('status is {}'.format(status))
print('price list is {}'.format(price))
print('current monowave label is {}'.format(idx))
print('current label is {}'.format(label))

status is inc
price list is [30598.0, 30776.0]
current monowave label is 1
current label is M1


In [174]:
#after add new price 
price.append(sim.gen())

#now price length is 3

if len(price)>=3:
    if status=='inc':
        if price[-2]<=price[-1]:
            label='M'+str(idx)
        else:
            idx+=1
            label='M'+str(idx)
            status='dec'
    elif status=='dec':
        if price[-2]<=price[-1]:
            idx+=1
            label='M'+str(idx)
            status='inc'
        else:
            label='M'+str(idx)

label_li.append(label)

In [175]:
# we can check all stuffs like price, idx, label, status
print('status is {}'.format(status))
print('price list is {}'.format(price))
print('current monowave label is {}'.format(idx))
print('current label is {}'.format(label))
print('cuurent label list is {}'.format(label_li))

status is dec
price list is [30598.0, 30776.0, 30748.79944116]
current monowave label is 2
current label is M2
cuurent label list is ['M1', 'M1', 'M2']


#### based on former tutorial, we defined function monoindexing.
*a few things are different with this tutorial but those are for another functions in the module so we skip them at this moment*

In [176]:
# hope u can define function based on the upper codes 
def mono_indexing(price,idx):
    '''
    price : list
    status : str
    idx : int
    label : str
    label should be saved on label_list
    '''
    label_li.append(label)
    return price,status,idx


- logger <br>
: it saves unique value of label and the price index of the label
~~(the name should be changed, i guess)~~

In [177]:
# why dont use set function of list 
# it change the order, so we choosed to define a new function,logger
# the current label is M2, so we start it from M1 
log_label=['M1']
log_index=[1]
if label not in log_label:
    log_label.append(label)
    log_index.append(len(price))

In [178]:
print('log index showes the start price idx+1 of monowave : {}'.format(log_index))
print('log_label showes unique element of label list : {}'.format(log_label))


log index showes the start price idx+1 of monowave : [1, 3]
log_label showes unique element of label list : ['M1', 'M2']


In [179]:
# if you want to get the starting price of monowave 
print(price)
print(label_li)
print('{} : M1 start price'.format(price[log_index[0]-1]))
print('{} : M2 start price'.format(price[log_index[-1]-1]))
# but I knew the 'index - 1' is very annoying after code became complicated

[30598.0, 30776.0, 30748.79944116]
['M1', 'M1', 'M2']
30598.0 : M1 start price
30748.79944116 : M2 start price


In [180]:
# hope u can define function based on the upper codes 
def logger(label):
    '''
    no return, but it have to save unique label and starting point
    '''
    

- Yes, we studied how to mark the each monowave based on their price, now we have to make some rules following an elliott wave book. each rule have to be activated based on a certain condition. <br>
 : ...

#### Rule of proportion
- this rule scan a direction of monowave in the given price list. 
- we calculate price change and price return if new monowave is detected. 

- How to find the new monowave's direction <br>
  i.e. if there are two monowaves, <br>
  mono_change : M1_end_price - M1_start_price <br>
  mono_return : M2_start_price - M1_end_price <br>
  
  after then, <br>
  if the return price is under 0.618 X the change price, <br> 
  **the M2 has direction** <br>
  **if not, the M2 has no direction.** <br> 

- How to find the last monowave's direction <br>
  calculate the angle of M1 with the initial price-time ratio,<br>
  if the angle is nearby 45, <br>
  **then M1 has direction** <br>
  (the rule allows upper_limit, lower_limit of angle) <br>

~~we have to add one more function that defines the end point of non-direction 1.618 * endprice `<` start price rule.~~
~~and also, considering the purpose of ROP, adding some function that make customer customize the ptr~~

In [181]:
# we gonna simulate 5 more simulation 
x=0
while x<5:
    #after add new price 
    price.append(sim.gen())

    if len(price)>=3:
        if status=='inc':
            if price[-2]<=price[-1]:
                label='M'+str(idx)
            else:
                idx+=1
                label='M'+str(idx)
                status='dec'
        elif status=='dec':
            if price[-2]<=price[-1]:
                idx+=1
                label='M'+str(idx)
                status='inc'
            else:
                label='M'+str(idx)

    label_li.append(label)

    if label not in log_label:
        log_label.append(label)
        log_index.append(len(price))
    x+=1

In [182]:
print('status is {}'.format(status))
print('price list is {}'.format(price))
print('current monowave label is {}'.format(idx))
print('current label is {}'.format(label))
print('cuurent label list is {}'.format(label_li)) 
print('current unique label is {}'.format(log_label))
print('cuurent monowave starting price list is {}'.format(log_index)) 

status is dec
price list is [30598.0, 30776.0, 30748.79944116, 30684.0, 30710.0, 30795.0, 30772.0, 30717.76945147]
current monowave label is 4
current label is M4
cuurent label list is ['M1', 'M1', 'M2', 'M2', 'M3', 'M3', 'M4', 'M4']
current unique label is ['M1', 'M2', 'M3', 'M4']
cuurent monowave starting price list is [1, 3, 5, 7]


In [186]:
#initial setting for rule of proportion 
timestamp=float(86400)
ratio=float(10/86400)
ptr=timestamp*ratio

direction=dict()


In [187]:
# lets do RofP for the M2
start_idx=log_index[0]-1
new_idx=log_index[1]-1
end_idx=new_idx-1

start_price=price[start_idx]
end_price=price[end_idx]
new_price=price[new_idx]

print('{} is the start price of M1, {} is the end price of M1, and {} is the new price of M2'.format(start_price,end_price,new_price))

30598.0 is the start price of M1, 30776.0 is the end price of M1, and 30748.79944116 is the new price of M2


In [188]:
mono_change=end_price-start_price
mono_return=new_price-end_price
mono_0618=abs(mono_change)*0.618

In [202]:
# How to find the last monowave's direction
time_pass=new_idx-start_idx
base_time=time_pass*ptr

#calculate angle with price change and base time
angle=round(math.degrees(math.atan(abs(mono_change)/base_time)),0)
print('angle is {}'.format(angle))

upper_limit=round(math.degrees(math.atan((base_time*1.25)/base_time)),0)
lower_limit=round(math.degrees(math.atan(base_time/(base_time*1.25))),0)

print('upper limit is {}'.format(upper_limit))
print('lower limit is {}'.format(lower_limit))

angle is 84.0
upper limit is 51.0
lower limit is 39.0


In [ ]:
if angle<upper_limit or angle>lower_limit:
    for key in range(start_idx,new_idx):
        direction.update({key:'Dir'})
    else:
        
        

In [203]:
# How to find the new monowave's direction
if mono_0618<abs(mono_return):
    direction.update({new_idx:'Non_Dir'})
else:
    direction.update({new_idx:'Dir'})

In [204]:
direction

{2: 'Dir'}

In [216]:
# for all labels, we could do this. 
for i in range(len(log_label)-1):
    start_idx=log_index[i]-1
    new_idx=log_index[i+1]-1
    end_idx=new_idx-1

    start_price=price[start_idx]
    end_price=price[end_idx]
    new_price=price[new_idx]

    mono_change=end_price-start_price
    mono_return=new_price-end_price
    mono0618=abs(mono_change)*0.618

    timepass=end_idx-start_idx
    base_time=timepass*ptr
    angle=round(math.degrees(math.atan(abs(mono_change)/base_time)),0)
    
    if mono_0618<abs(mono_return):
        direction.update({new_idx:'Non_Dir'})
    else:
        direction.update({new_idx:'Dir'})

    if angle<upper_limit and angle>lower_limit:
        for key in range(start_idx,new_idx):
            direction.update({key:'Dir'})

In [217]:
for key, value in sorted(direction.items(), key=lambda x: x[0]): 
    print("{} : {}".format(key, value))

0 : Dir
1 : Dir
2 : Dir
3 : Dir
4 : Dir
5 : Dir
6 : Dir


In [218]:
# we have to fix it .


#### Rule of Neurality
- this rule should be activated when the new label is updated.<br>
  if the angle between end price and new price is under 45, <br>
  the new monogram will roll back as prior monowave.<br>

- Rollback<br>
  delete updated log_label <br>
  delete updated log_index <br>
  delete the last label <br>
  add the label the same with prior label <br>
  subtract idx 1 <br>
  change status <br>

In [226]:
# for this rule, we restart the prior simulation.

sim(5570)

price=list()    # this is monowave price list
idx=1           # this is the number of current monowave 
status=str()    # it will be 'inc' or 'dec'
label_li=list()    # this is price's monowave label list

price.append(sim.gen())

if len(price)<3:
    label='M'+str(idx)
    if len(price)==2:
        if price[0] < price[-1]:
            status='inc'
        else:
            status='dec'
    else:
        pass
label_li.append(label)

#after add new price 
price.append(sim.gen())

#now price length is 2
if len(price)<3:
    label='M'+str(idx)
    if len(price)==2:
        if price[0] < price[-1]:
            status='inc'
        else:
            status='dec'
    else:
        pass
label_li.append(label)

#after add new price 
price.append(sim.gen())

#now price length is 3

if len(price)>=3:
    if status=='inc':
        if price[-2]<=price[-1]:
            label='M'+str(idx)
        else:
            idx+=1
            label='M'+str(idx)
            status='dec'
    elif status=='dec':
        if price[-2]<=price[-1]:
            idx+=1
            label='M'+str(idx)
            status='inc'
        else:
            label='M'+str(idx)

label_li.append(label)


In [227]:
log_label=['M1']
log_index=[1]
if label not in log_label:
    log_label.append(label)
    log_index.append(len(price))

In [229]:
# lets do RofP for the M2
new_idx=len(price)-1
new_price=price[new_idx]
end_price=price[new_idx-1]
change=new_price-end_price

angle=round(math.degrees(math.atan(abs(change)/ptr)),0)

if angle<45:
    print('rollback')
    log_label.pop()
    log_index.pop()
    label_li.pop()
    label_li.append(log_label[-1])
    idx-=1
    if status=='dec':
        status='inc'
    elif status=='inc':
        status='dec'

In [280]:
#execute this code when the new label is updated and then do next code
price.append(sim.gen())


if len(price)>=3:
    if status=='inc':
        if price[-2]<=price[-1]:
            label='M'+str(idx)
        else:
            idx+=1
            label='M'+str(idx)
            status='dec'
    elif status=='dec':
        if price[-2]<=price[-1]:
            idx+=1
            label='M'+str(idx)
            status='inc'
        else:
            label='M'+str(idx)

label_li.append(label)

if label not in log_label:
    log_label.append(label)
    log_index.append(len(price))

print(log_label)


['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16']


In [281]:
# lets do RofP for the M2
new_idx=len(price)-1
new_price=price[new_idx]
end_price=price[new_idx-1]
change=new_price-end_price

angle=round(math.degrees(math.atan(abs(change)/ptr)),0)
print(angle)
if angle<45:
    print('rollback')
    log_label.pop()
    log_index.pop()
    label_li.pop()
    label_li.append(log_label[-1])
    idx-=1
    if status=='dec':
        status='inc'
    elif status=='inc':
        status='dec'

# it activated when m16

39.0
rollback


In [283]:
log_label[-1]

'M15'

#### Rule of observation 

- This rule is activated when the new monowave label is updated

In [289]:
#intial setting for rule of observation 

group_monowave=dict()
group_idx=0

def group_wave_encoding(start,end,group_idx):
    # start and new is index number
    #print('start : {} end: {}'.format(start,end) )
    if start==end:
        end+=1
    idx_li=[x for x in range(start,end)]

    #print('this is start : {} and new : {}'.format(start,new))
    group_monowave[group_idx]=idx_li
    group_idx+=1
    tmp_r_ob=dict()
    print('{} : current group wave index elements is : {} '.format(group_idx-1,group_monowave[group_idx-1]))


In [293]:
tmp_r_ob=dict()

In [285]:
# for this rule, we restart the prior simulation.

sim(5570)

price=list()    # this is monowave price list
idx=1           # this is the number of current monowave 
status=str()    # it will be 'inc' or 'dec'
label_li=list()    # this is price's monowave label list

log_label=['M1']
log_index=[1]

price.append(sim.gen())

if len(price)<3:
    label='M'+str(idx)
    if len(price)==2:
        if price[0] < price[-1]:
            status='inc'
        else:
            status='dec'
    else:
        pass
label_li.append(label)

#after add new price 
price.append(sim.gen())

#now price length is 2
if len(price)<3:
    label='M'+str(idx)
    if len(price)==2:
        if price[0] < price[-1]:
            status='inc'
        else:
            status='dec'
    else:
        pass
label_li.append(label)



In [286]:

#after add new price 
price.append(sim.gen())

#now price length is 3

if len(price)>=3:
    if status=='inc':
        if price[-2]<=price[-1]:
            label='M'+str(idx)
        else:
            idx+=1
            label='M'+str(idx)
            status='dec'
    elif status=='dec':
        if price[-2]<=price[-1]:
            idx+=1
            label='M'+str(idx)
            status='inc'
        else:
            label='M'+str(idx)

label_li.append(label)

if label not in log_label:
    log_label.append(label)
    log_index.append(len(price))
    print(log_label)

In [287]:
print(log_label)

['M1', 'M2']


In [290]:
if len(log_label)==2:
    start=0
    end=len(label_li)-1

    group_idx=0
    group_wave_encoding(start,end,group_idx=group_idx)


0 : current group wave index elements is : [0, 1] 


In [305]:
#execute untill new label is updated

#after add new price 
price.append(sim.gen())

#now price length is 3

if len(price)>=3:
    if status=='inc':
        if price[-2]<=price[-1]:
            label='M'+str(idx)
        else:
            idx+=1
            label='M'+str(idx)
            status='dec'
    elif status=='dec':
        if price[-2]<=price[-1]:
            idx+=1
            label='M'+str(idx)
            status='inc'
        else:
            label='M'+str(idx)

label_li.append(label)

if label not in log_label:
    log_label.append(label)
    log_index.append(len(price))
    print(log_label)

['M1', 'M2', 'M3', 'M4']


In [306]:
if len(log_label)>2:
    mono_end_price=price[log_index[-1]-2]
    tmp_r_ob[mono_end_price]=log_index[-1]-2

In [299]:
group_idx=1

In [300]:
if group_idx<2:
    start_index=group_monowave[group_idx-1][0]
    start_price=price[start_index]
    end_index=group_monowave[group_idx-1][-1]
    end_price=price[end_index]
else:
    start_index=group_monowave[group_idx-2][-1]
    start_price=price[start_index]
    end_index=group_monowave[group_idx-1][-1]
    end_price=price[end_index]

In [301]:
mono_end_price=price[log_index[-1]-2]
#print('monogram{} endprice is {}'.format(label[-2],mono_end_price))
if start_price-end_price<0: # mono increased
    if mono_end_price>start_price and mono_end_price<end_price:

        pass
    elif mono_end_price<start_price:
        #new price : firstly inputted monowave's first price index
        #end price: lastly inputted monowave's end price index
        new_start_index=end_index+1
        new_end_index=log_index[-1]-1

        group_wave_encoding(new_start_index,new_end_index)
        ##print(new_start_index,new_end_index)


    elif mono_end_price>end_price:

    #    new price : firstly inputted monowave's first price index
    #    end price : minimum point of monowave index
        new_start_index=end_index+1
        new_end_index=tmp_r_ob[min(list(tmp_r_ob.keys()))]+1

        group_wave_encoding(new_start_index,new_end_index)
    #    new price 2: after point of end price index
    #    end price 2: lastly inpuuted monowave's end price index]
        new_start_index2=new_end_index
        new_end_index2=log_index[-1]-1

        group_wave_encoding(new_start_index2,new_end_index2)

elif start_price-end_price>0: # mono decreased
    if mono_end_price<start_price and mono_end_price>end_price:

        pass
    elif mono_end_price>start_price:
        #new price : firstly inputted monowave's first price index
        #end price: lastly inputted monowave's end price index
        new_start_index=end_index+1
        new_end_index=log_index[-1]-1

        group_wave_encoding(new_start_index,new_end_index)

    elif mono_end_price<end_price:
    #    new price : firstly inputted monowave's first price index
    #    end price : minimum point of monowave index
        new_start_index=end_index+1
        new_end_index=tmp_r_ob[max(list(tmp_r_ob.keys()))]+1

        group_wave_encoding(new_start_index,new_end_index)
    #    new price 2: after point of end price index
    #    end price 2: lastly inpuuted monowave's end price index]
        new_start_index2=new_end_index
        new_end_index2=log_index[-1]-1

        group_wave_encoding(new_start_index2,new_end_index2)

In [303]:
print(group_monowave)

{0: [0, 1]}
